In [1]:
from mindquantum.core.circuit import Circuit, UN, add_suffix, apply, decompose_single_term_time_evolution
from mindquantum.core.gates import H, Rzz, RX, Rzz,RZ, RY, Measure, BarrierGate
from mindquantum.core.operators import Hamiltonian, QubitOperator
from mindquantum.framework import MQAnsatzOnlyLayer
from mindquantum.simulator import Simulator
from mindquantum.core.operators import commutator
import time
from mindquantum.core.parameterresolver import ParameterResolver
from mindquantum import *

import mindspore.nn as nn
from mindspore import Tensor
import mindspore as ms
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="CPU")

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import matplotlib.pylab as pylab
from tqdm import tqdm

from itertools import combinations
from collections import OrderedDict as ordict

## Build quantum circuits

In [2]:
def param2dict(keys, values):
    # make the parameters and coresponding label to a dict
    param_dict = {}
    for (key, value) in zip(keys, values):
        param_dict[key] = value
    return param_dict

def get_expectation_of_hamitonian(circ, ham, pr):
    # calculate the expectation of hamitonian
    sim = Simulator('mqvector', circ.n_qubits)
    sim.apply_circuit(circ, pr)
    result = sim.get_expectation(ham)
        
    return result.real

def E0_energy(ham_operator):
    
    # 首先根据哈密顿量的算符将其转化为矩阵形式。
    ham_matrix = ham_operator.matrix()
    ham_matrix = ham_matrix.todense()

    # calculate the eigenvalue and eigenvector
    eigval, eigvec = np.linalg.eig(ham_matrix)

    # ground energy
    E0 = min(eigval).real
    #print(eigval, eigvec)
    
    return E0

def train(ham, ansatze, iteration):

    # bulid the quantum circuit
    
    circ = ansatze                                         # 将初始化线路与ansatz线路组合成一个线路
    sim = Simulator('mqvector', ansatze.n_qubits)  
    #sim.set_qs(initial_ground_state)
    
    grad_ops = sim.get_expectation_with_grad(ham, circ)    # 获取计算变分量子线路的期望值和梯度的算子
    net = MQAnsatzOnlyLayer(grad_ops)                             
    opti = nn.Adam(net.trainable_params(), learning_rate=0.05)     
    train_net = nn.TrainOneStepCell(net, opti)                    

    result = []
    #from tqdm import tqdm
    for i in range(iteration):
        train_net() 
        result.append(np.array(train_net()[0]))
    pr = dict(zip(ansatze.params_name, net.weight.asnumpy())) # 获取线路参数
        
    return pr, result


def state_sampling(ansatze, pr):
    
    shots = 1024

    for i in range(ansatze.n_qubits):
        ansatze += Measure().on(i)
    

    sim = Simulator('mqvector', ansatze.n_qubits)
    #sim.set_qs(initial_ground_state)
    res = sim.sampling(ansatze, pr, shots=shots)
    
    return res

def CD_operator(CD):
    
    CD_term = []
    for cd in CD.split():
        CD_term.append(cd[1])
    k = 0
    out = ordict()
    for cd in CD_term:
        for key, term in cd.terms.items():
            if len(key) == 2:
                #print(len(key))
                if key not in out:
                    out[key] = ordict({f"p{k}": 1})
                    k += 1
    return out

def CD_ansataz(out):
    "Transform a pauli ansatz to parameterized quantum circuit."

    circuit = Circuit()
    # circuit += inital_circuits(occupied, circuit_description)
        
    z = 0
    for k, v in out.items():
        index1 = (k[0][0], k[1][0])
        index2 = (k[1][0], k[0][0])

        pauli_operator = (str(k[0][1]), str(k[1][1]))
        if pauli_operator == ('Z','Y'):
            circuit += Ryz(f'{z}').on(index2)          
            z+=1
    # for k, v in out.items():
    #     circuit += decompose_single_term_time_evolution(k, v)
    for i in range(n_items):
        circuit += RY(f'ry{i}').on(i)
    
    return circuit

### The Hamitonian 
$H_p = \sum_{i=1}^{n}(-\frac{\alpha}{2}+\beta(C-\frac{1}{2}\sum_{i}^{n}\omega_{i}))Z_{i}\omega_{i} + \sum_{i<j}\frac{\beta}{2}\omega_{i}\omega_{j}Z_{i}Z_{j}.$

In [3]:
def generate_h_one_body(n, W, alpha, beta, capacity):
    ele1 = QubitOperator()
    for i in range(n):
            ele1 += QubitOperator(f'Z{i}',-W[i]*(alpha/2 - beta*capacity))
    for i in range(n):
        for j in range(n):
            ele1 += QubitOperator(f'Z{i}',-1/2*beta*W[i]*W[j])
    return ele1

def generate_h_two_body(n, W, beta):
    ele1 = QubitOperator()
    h_two_body = QubitOperator()
    for i in range(n):
        for j in range(n):
            if i < j:
                ele1 += QubitOperator(f'Z{i} Z{j}',W[i]*W[j]*beta/2)
    return ele1

def generate_h_mixer(n):
    h_mixer = QubitOperator()
    for i in range(n):
        h_mixer += QubitOperator(f'X{i}')
    return h_mixer

In [4]:
# Function of generating all possible solution in one bin
def find_combinations(elements, target_sum):
    result = []
    # Enumerate the elements to keep track of their indices
    enumerated_elements = list(enumerate(elements))
    for r in range(1, len(elements) + 1):
        for combo in combinations(enumerated_elements, r):
            # Calculate the sum of the current combination
            combo_sum = sum(ele[1] for ele in combo)
            if combo_sum <= target_sum:
                # Store the indices of the elements in the combination
                result.append([ele[0] for ele in combo])
    return result


# Function of calculating the minimum positive difference of weights
def min_positive_difference(lst):
    # Sort the list in ascending order
    sorted_lst = sorted(lst)
    
    # Initialize min_diff to a large value
    min_diff = float('inf')

    # Iterate through the sorted list to find the minimum positive difference
    for i in range(len(sorted_lst) - 1):
        diff = sorted_lst[i + 1] - sorted_lst[i]
        if 0 < diff < min_diff:
            min_diff = diff

    return min_diff if min_diff != float('inf') else None

### check the minimum number of bin

In [5]:
def find_min_number(lists, target):
    opti_solu = 0
    for n in tqdm(range(n_items)):
        nested_list = list(combinations(lists,n))
        for sublist in nested_list:
            l = []
            for i in sublist:
                l+=i
                l = sorted(l)
                if l==target:
                    opti_solu += 1
        if opti_solu != 0:
            return n, opti_solu
            break
    return 999, 999

### define the traing process, including optimizer, iteration

In [6]:
def Quantum_cd_mixer(h_m, ci, n_layer, n_items):
       
    u_m = 0
    index=0
    for i in h_m:
        u_m += decompose_single_term_time_evolution(i, ParameterResolver(f'c{index}'))
        index=index+1

    p = n_layer
    prep_circ = UN(H, n_items)
    ansatz_template =   u_m + ci #u_p +
    ansatz = Circuit() + prep_circ

    for i in range(p):
        ansatz += add_suffix(ansatz_template, str(i)) + BarrierGate()
        
    return ansatz

def Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items):
    
            
    u_m = 0
    index=0
    for i in h_m:
        u_m += decompose_single_term_time_evolution(i, ParameterResolver(f'b{index}'))
        index=index+1

    u_p = 0
    index=0
    for i in h_p:
        u_p += decompose_single_term_time_evolution(i, ParameterResolver(f'a{index}'))
        index=index+1


    p = n_layer
    prep_circ = UN(H, n_items)
    ansatz_template =   u_p + u_m 
    ansatz = Circuit() + prep_circ

    for i in range(p):
        ansatz += add_suffix(ansatz_template, str(i)) + BarrierGate()
        
    return ansatz

In [7]:
def data_analyze(n_items, solution_set, W, Capa, target_solu):
    
    # Extracting keys with values greater than 100
    data = [key for sub_dict in solution_set for key, value in sub_dict.items()]
    # Remove the duplicate keys
    result_list = list(set(data))
    # Calculate the weight sum for each binary string
    weight_sums = [sum(W[i] for i in range(len(result_list_sub)) if result_list_sub[i] == '1') for result_list_sub in result_list]
    # Combine the binary strings and their corresponding weight sums
    combined_data = list(zip(result_list, weight_sums))
    # Filter out the empty lists
    filtered_empty = [(binary_string, value) for binary_string, value in combined_data if value != 0]
    # Filter out the data whose weight sum is bigger than Capacity
    filtered_data = [(binary_string, weight_sum) for binary_string, weight_sum in filtered_empty if weight_sum <= Capa]
    # Sort the combined data by weight sum in descending order
    sorted_data = sorted(filtered_data, key=lambda x: x[1], reverse=True)
    # Read the feasible solutions
    sorted_binary_strings = [data[0] for data in sorted_data] 
    
    ar = len(sorted_binary_strings)/len(target_solu)
    sq = len(sorted_binary_strings)/len(filtered_empty)

    return  sorted_binary_strings, ar, sq

In [8]:
dk = 2.5
ite = 100
#n_layer = 1

#dk_list = [1, 1.5, 2, 2.5, 3, 3.5, 4]

#iteration_list = [100, 150, 200]
n_layer_list = [1]


In [9]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [42, 44, 63, 41, 51, 45, 42, 69, 50, 40]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [10]:
# Record the current time before the first process

batch = 5
sr_set = np.zeros((len(n_layer_list), batch))
sq_set = np.zeros((len(n_layer_list), batch))
minbin_set = np.zeros((len(n_layer_list), batch))
n_solu_set = np.zeros((len(n_layer_list), batch))

cost_fun = []

for i_index, n_layer in enumerate(n_layer_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality = data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        #print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        #print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 50%|█████     | 5/10 [00:01<00:01,  3.86it/s]


successful ratio is:  0.7962962962962963
solution quality is:  0.8431372549019608
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  4.62it/s]


successful ratio is:  0.7592592592592593
solution quality is:  0.803921568627451
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  3.40it/s]


successful ratio is:  0.8148148148148148
solution quality is:  0.7719298245614035
minimum number of bin is 5


 50%|█████     | 5/10 [00:00<00:00,  5.32it/s]


successful ratio is:  0.7407407407407407
solution quality is:  0.7547169811320755
minimum number of bin is 5


 50%|█████     | 5/10 [00:00<00:00,  7.18it/s]

successful ratio is:  0.7037037037037037
solution quality is:  0.7037037037037037
minimum number of bin is 5


In [36]:
# # Write data to a text file
# with open('qaoa-la-W1.txt', 'w') as f:
#     f.write("layer, batch, sr_set, sq_set, minbin_set, n_solu\n")
#     for i_index, i in enumerate(n_layer_list):
#         for j in range(batch):
#             f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [11]:
circ.summary()


====================================Circuit Summary====================================
|Total number of gates  : 346.                                                        |
|Parameter gates        : 65.                                                         |
|with 65 parameters are :                                                             |
|a0_0, a1_0, a2_0, a3_0, a4_0, a5_0, a6_0, a7_0, a8_0, a9_0..                        .|
|Number qubit of circuit: 10                                                          |


In [45]:
circ.svg().to_file('qaoa_ansatz.svg')

In [11]:
dk = 2.5
ite = 100
#n_layer = 1

#dk_list = [1, 1.5, 2, 2.5, 3, 3.5, 4]

#iteration_list = [100, 150, 200]
n_layer_list = [1]

In [14]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [55, 59, 52, 46, 40, 55, 79, 40, 55, 56]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [15]:
# Record the current time before the first process

batch = 5
sr_set = np.zeros((len(n_layer_list), batch))
sq_set = np.zeros((len(n_layer_list), batch))
minbin_set = np.zeros((len(n_layer_list), batch))
n_solu_set = np.zeros((len(n_layer_list), batch))

for i_index, n_layer in enumerate(n_layer_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality = data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        #print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        #print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 50%|█████     | 5/10 [00:00<00:00,  8.75it/s]


successful ratio is:  0.7708333333333334
solution quality is:  0.9024390243902439
minimum number of bin is 5


 50%|█████     | 5/10 [00:00<00:00, 10.18it/s]


successful ratio is:  0.7291666666666666
solution quality is:  0.8974358974358975
minimum number of bin is 5


 50%|█████     | 5/10 [00:01<00:01,  4.95it/s]


successful ratio is:  0.8333333333333334
solution quality is:  0.8695652173913043
minimum number of bin is 5


 69%|██████▉   | 33/48 [00:13<00:05,  2.51it/s]


KeyboardInterrupt: 

In [14]:
# Write data to a text file
with open('qaoa-la-W2.txt', 'w') as f:
    f.write("layer, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(n_layer_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [19]:
dk = 2.5
ite = 100
#n_layer = 1

#dk_list = [1, 1.5, 2, 2.5, 3, 3.5, 4]

#iteration_list = [100, 150, 200]
n_layer_list = [1]


In [20]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [28, 30, 31, 36, 46, 32, 49, 29, 33, 21]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [21]:
# Record the current time before the first process

batch = 5
sr_set = np.zeros((len(n_layer_list), batch))
sq_set = np.zeros((len(n_layer_list), batch))
minbin_set = np.zeros((len(n_layer_list), batch))
n_solu_set = np.zeros((len(n_layer_list), batch))

cost_fun = []

for i_index, n_layer in enumerate(n_layer_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality = data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        #print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        #print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

100%|██████████| 48/48 [00:19<00:00,  2.42it/s]


[('0100000110', 94), ('0100100000', 65), ('1000010101', 126), ('0000001101', 95), ('1100100000', 86), ('0101000100', 113), ('1001000001', 98), ('0000001010', 66), ('1001000000', 70), ('0000010111', 135), ('0001100000', 81), ('1000100101', 112), ('0000010000', 46), ('1000010000', 67), ('1010001000', 86), ('0100001010', 99), ('0000010010', 76), ('0000000001', 28), ('0000000111', 89), ('1011000000', 99), ('1010000000', 50), ('0100001001', 97), ('0101000010', 112), ('1010001001', 114), ('1110100000', 115), ('0000000100', 31), ('1000000010', 51), ('0000011100', 113), ('0100010100', 110), ('1100100010', 116), ('0000110001', 106), ('0010010010', 105), ('0001001000', 85), ('0000100000', 32), ('1000010011', 125), ('1000000001', 49), ('1000101001', 117), ('1000101100', 120), ('0101000000', 82), ('1100001010', 120), ('0010010001', 103), ('1001100100', 133), ('0100101000', 101), ('1000110000', 99), ('1010010000', 96), ('1010100001', 110), ('0010000111', 118), ('0010000000', 29), ('0010100000', 61)

 30%|███       | 3/10 [00:01<00:04,  1.75it/s]


successful ratio is:  0.5756097560975609
solution quality is:  0.9076923076923077
minimum number of bin is 3


100%|██████████| 48/48 [00:19<00:00,  2.50it/s]


...

successful ratio is:  0.4634146341463415
solution quality is:  0.979381443298969
minimum number of bin is 3


100%|██████████| 48/48 [00:46<00:00,  1.03it/s]


[('0100100000', 65), ('0011100000', 110), ('0000110000', 78), ('0000001101', 95), ('1100100000', 86), ('0101000100', 113), ('0001010010', 125), ('0010000100', 60), ('0000001010', 66), ('1001000000', 70), ('0011000010', 108), ('0001100000', 81), ('1000100101', 112), ('0000010000', 46), ('1010001000', 86), ('1011000000', 99), ('1010000000', 50), ('0101000010', 112), ('0000000100', 31), ('1000000010', 51), ('0100010100', 110), ('0100010000', 79), ('0000110001', 106), ('0010010010', 105), ('0110000001', 90), ('1111000000', 132), ('0000001011', 94), ('0001001000', 85), ('0000101001', 96), ('1100000101', 113), ('0000100000', 32), ('0001100010', 111), ('1000000001', 49), ('0100000101', 92), ('0100000010', 63), ('0110000100', 93), ('0101000000', 82), ('1100000100', 85), ('1010010010', 126), ('0001000010', 79), ('0001100001', 109), ('1000110000', 99), ('1010010000', 96), ('1000101000', 89), ('1001000010', 100), ('0010000000', 29), ('0010000001', 57), ('0100100100', 96), ('1000001000', 57), ('10

 30%|███       | 3/10 [00:00<00:00, 28.01it/s]

successful ratio is:  0.5365853658536586
solution quality is:  0.9734513274336283
minimum number of bin is 3


In [22]:
# Write data to a text file
with open('qaoa-la-W3.txt', 'w') as f:
    f.write("layer, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(n_layer_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [24]:
dk = 2.5
ite = 100
#n_layer = 1

#dk_list = [1, 1.5, 2, 2.5, 3, 3.5, 4]

#iteration_list = [100, 150, 200]
n_layer_list = [1, 2, 3, 4]

In [25]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [41, 31, 30, 26, 21, 24, 31, 30, 42, 29]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [26]:
# Record the current time before the first process

batch = 5
sr_set = np.zeros((len(n_layer_list), batch))
sq_set = np.zeros((len(n_layer_list), batch))
minbin_set = np.zeros((len(n_layer_list), batch))
n_solu_set = np.zeros((len(n_layer_list), batch))

for i_index, n_layer in enumerate(n_layer_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality = data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        #print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        #print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 30%|███       | 3/10 [00:00<00:00, 16.12it/s]


successful ratio is:  0.49433962264150944
solution quality is:  0.8506493506493507
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:02,  3.44it/s]


successful ratio is:  0.5018867924528302
solution quality is:  0.8926174496644296
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 15.70it/s]


...

successful ratio is:  0.4377358490566038
solution quality is:  0.943089430894309
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 17.06it/s]


successful ratio is:  0.47547169811320755
solution quality is:  0.9197080291970803
minimum number of bin is 3


 30%|███       | 3/10 [00:00<00:00, 31.48it/s]

successful ratio is:  0.39245283018867927
solution quality is:  0.9719626168224299
minimum number of bin is 3


In [27]:
# Write data to a text file
with open('qaoa-la-W4.txt', 'w') as f:
    f.write("layer, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(n_layer_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [28]:
dk = 2.5
ite = 100
#n_layer = 1

#dk_list = [1, 1.5, 2, 2.5, 3, 3.5, 4]
#iteration_list = [100, 150, 200]
n_layer_list = [1, 2, 3, 4]

In [29]:
n_items = 10 #number of items
capacity = 120 #maximum capacity of box

# insert QAOA weight list to compare fairly
W = [42, 55, 52, 80, 55, 80, 53, 46, 79, 66]

#np.random.normal(loc=capacity/2, size=n)
beta = np.min(W)/5
delta_w = min_positive_difference(W)

# generate exact solutions
result_combinations = find_combinations(W, capacity)
target_solu = list(result_combinations)
target = list(range(n_items))

In [30]:
# Record the current time before the first process

batch = 5
sr_set = np.zeros((len(n_layer_list), batch))
sq_set = np.zeros((len(n_layer_list), batch))
minbin_set = np.zeros((len(n_layer_list), batch))
n_solu_set = np.zeros((len(n_layer_list), batch))

for i_index, n_layer in enumerate(n_layer_list):

    for j in range(batch):

        solution_set = []
        # qu_time = []
        # start_time_process1 = time.time()

        for k in tqdm(np.arange(1,(capacity/delta_w),dk)):
            alpha = 2*beta*(capacity-k*delta_w)
            h_p = generate_h_two_body(n_items, W, beta) + generate_h_one_body(n_items, W, alpha, beta, capacity)
            h_m = generate_h_mixer(n_items)
            h_cd = commutator(h_m,h_p)#CD pool
            cd = CD_operator(h_cd)
            ci = CD_ansataz(cd)

            circ = Quantum_QAOA_Circuits(h_m, h_p, n_layer, n_items)
            #ground_energy.append(E0_energy(h_p))

            ham = Hamiltonian(h_p)
            sim = Simulator('mqvector', n_items)
            grad_ops = sim.get_expectation_with_grad(ham, circ)

            pr, result = train(ham, circ, ite)
            #cost_fun.append(result)
            res = state_sampling(circ, pr)
            solution_set.append(res.data)

        # Record the current time after the first process
        # end_time_process1 = time.time()

        # Calculate the elapsed time for the quantum process
        # elapsed_time_process1 = end_time_process1 - start_time_process1
        # qu_time[i_index, j] = elapsed_time_process1

        elements_fea, succ_ratio, solution_quality = data_analyze(n_items, solution_set, W[::-1], capacity, target_solu)
        positions = [[n_items-1-i for i, char in enumerate(element) if char == '1'] for element in elements_fea]
        minbin, n_solu = find_min_number(positions,target)
        
        minbin_set[i_index, j] = minbin
        n_solu_set[i_index, j] = n_solu
        sr_set[i_index, j] = succ_ratio
        sq_set[i_index, j] = solution_quality

        # Record the current time before the second process
        # end_time_process2 = time.time()

        # Calculate the elapsed time for the classic process
        # elapsed_time_process2 = end_time_process2 - end_time_process1
        # cl_time[i_index, j] = elapsed_time_process2

        print('successful ratio is: ',succ_ratio)
        print('solution quality is: ',solution_quality)
        print('minimum number of bin is',minbin)
        #print("Elapsed time for quantum process:", np.mean(elapsed_time_process1), "seconds")
        #print("Elapsed time for classic process:", np.mean(elapsed_time_process2), "seconds")

 70%|███████   | 7/10 [00:01<00:00,  6.98it/s]


successful ratio is:  0.896551724137931
solution quality is:  0.7222222222222222
minimum number of bin is 7


 70%|███████   | 7/10 [00:01<00:00,  6.99it/s]


successful ratio is:  0.896551724137931
solution quality is:  0.7428571428571429
minimum number of bin is 7


 70%|███████   | 7/10 [00:05<00:02,  1.39it/s]


...

successful ratio is:  1.0
solution quality is:  0.7837837837837838
minimum number of bin is 7


 70%|███████   | 7/10 [00:00<00:00,  9.47it/s]


successful ratio is:  0.8620689655172413
solution quality is:  0.6944444444444444
minimum number of bin is 7


 70%|███████   | 7/10 [00:01<00:00,  4.78it/s]

successful ratio is:  0.9310344827586207
solution quality is:  0.6428571428571429
minimum number of bin is 7


In [31]:
# Write data to a text file
with open('qaoa-la-W5.txt', 'w') as f:
    f.write("layer, batch, sr_set, sq_set, minbin_set, n_solu\n")
    for i_index, i in enumerate(n_layer_list):
        for j in range(batch):
            f.write(f"{i}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {minbin_set[i_index, j]}, {n_solu_set[i_index, j]}\n")

In [32]:
circ.summary()

====================================Circuit Summary====================================
|Total number of gates  : 1364.                                                       |
|Parameter gates        : 260.                                                        |
|with 260 parameters are:                                                             |
|a0_0, a1_0, a2_0, a3_0, a4_0, a5_0, a6_0, a7_0, a8_0, a9_0..                        .|
|Number qubit of circuit: 10                                                          |


In [20]:
# Write data to a text file
with open('0412-qaoa-n_layer-dk=1.txt', 'w') as f:
    f.write("dk, batch, sr_set, sq_set, cost_fun, minbin_set, qu_time, cl_time\n")
    for i_index, dk in enumerate(n_layer_list):
        for j in range(batch):
            f.write(f"{dk}, {j}, {sr_set[i_index, j]}, {sq_set[i_index, j]}, {cost_fun[i_index, j]}, {minbin_set[i_index, j]}, {qu_time[i_index, j]}, {cl_time[i_index, j]}\n")

In [14]:
print('p =',n_layer)
print('dk =',dk)
print('Iteration =',ite)

p = 3
dk = 3
Iteration = 100


In [15]:
print('- cd-minbin_set:',minbin_set)
print('- cd-sr_set:',sr_set)
print('- cd-sq_set:',sq_set,'\n')
print("**Elapsed time quantum:", np.mean(elapsed_time_process1), "seconds**")
print("**Elapsed time classic:", np.mean(elapsed_time_process2), "seconds**")



- cd-minbin_set: [[6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]]
- cd-sr_set: [[0.88888889 0.88888889 0.86111111 0.83333333 0.91666667 0.91666667
  0.88888889 0.91666667 0.94444444 0.97222222]
 [0.91666667 0.97222222 1.         0.91666667 1.         0.97222222
  0.88888889 0.94444444 0.94444444 0.94444444]
 [0.94444444 0.94444444 0.94444444 0.88888889 0.94444444 0.94444444
  0.94444444 0.97222222 0.88888889 0.88888889]]
- cd-sq_set: [[0.69565217 0.66666667 0.64583333 0.6122449  0.67346939 0.64705882
  0.61538462 0.64705882 0.66666667 0.66037736]
 [0.58928571 0.63636364 0.61016949 0.57894737 0.62068966 0.63636364
  0.61538462 0.60714286 0.64150943 0.60714286]
 [0.64150943 0.62962963 0.59649123 0.59259259 0.55737705 0.59649123
  0.5862069  0.61403509 0.60377358 0.62745098]] 

**Elapsed time quantum: 38.01597476005554 seconds**
**Elapsed time classic: 84.73806428909302 seconds**


In [16]:
print('- cd-minbin_set:',minbin_set)
print('- cd-sr_set:',sr_set)
print('- cd-sq_set:',sq_set)
print("**Elapsed time:", elapsed_time, "seconds**")

- cd-minbin_set: [[6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]
 [6. 6. 6. 6. 6. 6. 6. 6. 6. 6.]]
- cd-sr_set: [[0.88888889 0.88888889 0.86111111 0.83333333 0.91666667 0.91666667
  0.88888889 0.91666667 0.94444444 0.97222222]
 [0.91666667 0.97222222 1.         0.91666667 1.         0.97222222
  0.88888889 0.94444444 0.94444444 0.94444444]
 [0.94444444 0.94444444 0.94444444 0.88888889 0.94444444 0.94444444
  0.94444444 0.97222222 0.88888889 0.88888889]]
- cd-sq_set: [[0.69565217 0.66666667 0.64583333 0.6122449  0.67346939 0.64705882
  0.61538462 0.64705882 0.66666667 0.66037736]
 [0.58928571 0.63636364 0.61016949 0.57894737 0.62068966 0.63636364
  0.61538462 0.60714286 0.64150943 0.60714286]
 [0.64150943 0.62962963 0.59649123 0.59259259 0.55737705 0.59649123
  0.5862069  0.61403509 0.60377358 0.62745098]]


NameError: name 'elapsed_time' is not defined

In [ ]:
gs = E0_energy(h_p)

In [ ]:
plt.xlabel('iteration', fontsize=12)
plt.ylabel('Energy', fontsize=12)
plt.title("100 iteration", fontsize=12)
plt.axhline(y=gs, color="black", linestyle="--", label="True energy")
plt.plot(result)
plt.savefig('100-dc-p=1.png')

In [ ]:
yz = QubitOperator(f'Y0 Z1',1)
c = Circuit()
c += decompose_single_term_time_evolution(yz, ParameterResolver(f'b{index}'))

In [ ]:
c.svg().to_file('yz_0314.svg')